In [1]:
import scipy.io
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
import torch.optim as optim

In [2]:
data = scipy.io.loadmat("../data/ex4data1.mat")
raw_X = data['X']
raw_y = data['y']
raw_y[raw_y == 10] = 0

x_train, x_test, y_train, y_test = train_test_split(raw_X, raw_y, test_size=0.2, random_state=42)

In [3]:
class CustomDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.int64).squeeze()
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        if self.transform:
            feature = self.transform(feature)
        return feature, label


train_dataset = CustomDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = CustomDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


for X, y in train_loader:
    print(f"Feature batch shape: {X.size()}")
    print(f"Labels batch shape: {y.size()}")
    break

Feature batch shape: torch.Size([64, 400])
Labels batch shape: torch.Size([64])


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SimplifiedVGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(SimplifiedVGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # conv3-128
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 5 * 5, 100),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(100, num_classes),
        )

        for layer in self.features:
            if isinstance(layer, nn.Conv2d):
                nn.init.kaiming_normal_(layer.weight, mode='fan_out', nonlinearity='relu')
                if layer.bias is not None:
                    nn.init.constant_(layer.bias, 0)

        for layer in self.classifier:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight, gain=1.414)
                nn.init.constant_(layer.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)
        return x

model = SimplifiedVGG16().to(device)
print(model)

SimplifiedVGG16(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=3200, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=100, out_features=10, bias=True)
  )
)


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for batch_index, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        inputs = inputs.view(-1, 1, 20, 20)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_index + 1) % 20 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_index+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [20/63], Loss: 1.4873
Epoch [1/5], Step [40/63], Loss: 0.8147
Epoch [1/5], Step [60/63], Loss: 0.5311
Epoch [2/5], Step [20/63], Loss: 0.4353
Epoch [2/5], Step [40/63], Loss: 0.4637
Epoch [2/5], Step [60/63], Loss: 0.1728
Epoch [3/5], Step [20/63], Loss: 0.2771
Epoch [3/5], Step [40/63], Loss: 0.2850
Epoch [3/5], Step [60/63], Loss: 0.2415
Epoch [4/5], Step [20/63], Loss: 0.1831
Epoch [4/5], Step [40/63], Loss: 0.1944
Epoch [4/5], Step [60/63], Loss: 0.1696
Epoch [5/5], Step [20/63], Loss: 0.1484
Epoch [5/5], Step [40/63], Loss: 0.0875
Epoch [5/5], Step [60/63], Loss: 0.1464


In [6]:
model.eval()
test_loss, correct = 0, 0
size = len(test_loader.dataset)
num_batches = len(test_loader)
with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        X = X.view(-1, 1, 20, 20)
        pred = model(X)
        test_loss += criterion(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n  Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Test Error: 
  Accuracy: 96.8%, Avg loss: 0.116673 

